This is a notebook used to prepare data for pH prediction model.
Requires raw data containting rcsb id, sequence and pH downloaded using the load_data module.
Also requires xcf folder to be added to PYTHONPATH

In [1]:
import xcf
import numpy as np
import pandas as pd
from datetime import date

In [2]:
#Paths to input raw data .csv file and output folder
data_raw_filename = '../data/raw/data_raw_2022-11-06.csv'
output_folder = '../data/interim/'

data_raw = pd.read_csv(data_raw_filename)
data_raw.head()

,Unnamed: 0,matthews,percent_solvent,rcsb_id,method,pH,conditions,temp,resolution,organism,sequence
0,0,3.09,60.20,101M,NaN,9.0,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",NaN,2.07,Physeter catodon,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
1,1,2.75,55.28,102L,NaN,9.0,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",NaN,1.74,Escherichia virus T4,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNAAAKSE...
2,2,3.09,60.20,102M,NaN,9.0,"3.0 M AMMONIUM SULFATE, 20 MM TRIS, 1MM EDTA, ...",NaN,1.84,Physeter catodon,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...
3,3,2.70,54.46,103L,NaN,7.0,"3.0 M AMMONIUM SULFATE, UNBUFFERED, pH 7.0",NaN,1.90,Escherichia virus T4,MNIFEMLRIDEGLRLKIYKDTEGYYTIGIGHLLTKSPSLNSLDAAK...
4,4,3.09,60.30,103M,NaN,9.0,"3.0 M AMMONIUM SULFATE, UNBUFFERED, pH 9.0",NaN,2.07,Physeter catodon,MVLSEGEWQLVLHVWAKVEADVAGHGQDILIRLFKSHPETLEKFDR...


One-hot encoding test for first 10 raw entries.

In [3]:
data_clean_test = xcf.process_data(data_raw[0:10], mode = 'pHpred')
print(data_clean_test.memory_usage(deep=True))
print(data_clean_test['seq_one_hot'][0])

Index             610
pH                 80
sequence         2154
len_seq            80
seq_one_hot    157360
dtype: int64
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]]


Looks OK, running for the whole dataset. Took ~1min on my PC.

In [4]:
data_clean = xcf.process_data(data_raw, mode = 'pHpred')
print(data_clean.memory_usage(deep=True))

Index             5226602
pH                 685456
sequence         29327594
len_seq            685456
seq_one_hot    1348291952
dtype: int64


Save interim data. Numpy ndarrays are converted to a list of lists for saving as parquet. Crashes on full data set, borrowed the method of doing it in batches form loda_data.py. Still crashes. Maybe it's better to one hot encode each time before using the data???

In [6]:
today = date.today()
today = today.isoformat()
batch_size = 10000
n_batches = len(data_clean)//batch_size
data_for_saving = data_clean

for i in range(n_batches+1):
    data_for_saving['seq_one_hot'].iloc[batch_size*i:batch_size*i+batch_size]=data_for_saving['seq_one_hot'].iloc[batch_size*i:batch_size*i+batch_size].apply(lambda X: X.tolist())
    
with open(f'{output_folder}data_pHpred_interim_{today}.parquet', mode = 'wb') as out_file:
    data_for_saving.to_parquet(out_file)

/tmp/ipykernel_371/2796790584.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_for_saving['seq_one_hot'].iloc[batch_size*i:batch_size*i+batch_size]=data_for_saving['seq_one_hot'].iloc[batch_size*i:batch_size*i+batch_size].apply(lambda X: X.tolist())


: 

: 